https://github.com/yoongi0428/RecSys_PyTorch/tree/4787f1a69d56664a34512eced6909dfab85cc07b

In [24]:
from evaluation.backend.python.holdout import *
from evaluation.backend.python.loo import *

In [6]:
import pandas as pd
import numpy as np

df_pred = pd.read_csv('../pred.csv')
df_test = pd.read_csv('../test.csv')

df_pred_new = pd.merge(
    df_pred, 
    df_test.loc[:, ['user_id', 'item_id', 'relevance']], on=['user_id', 'item_id'], how='left'
)

df_pred_new = df_pred_new[df_pred_new.user_id.isin(df_test.user_id)]

df_pred_new.fillna(0, inplace=True)

df_pred_new.loc[df_pred_new.relevance != 0, 'relevance'] = 1

df_pred_new.relevance.value_counts()

0.0    51516
1.0     3144
Name: relevance, dtype: int64

In [11]:
recs = df_pred_new.groupby('user_id').item_id.apply(list).sort_index().tolist()

In [22]:
from collections import OrderedDict

gt_recommendations = OrderedDict()

for user in sorted(df_pred_new.user_id.unique()):
    gt_recommendations[user] = df_test[df_test.user_id == user].item_id.tolist()

In [23]:
compute_holdout_metrics_py(recs, gt_recommendations, ks=[20])

OrderedDict([('Prec', {20: Prec@20: mean=0.0575, std=0.0918}),
             ('Recall', {20: Recall@20: mean=0.0963, std=0.1720}),
             ('NDCG', {20: NDCG@20: mean=0.0933, std=0.1427})])

In [25]:
compute_loo_metrics_py(recs, gt_recommendations, ks=[20])

IndexError: index 0 is out of bounds for axis 0 with size 0